# Setup Student Roster
For importing into Google Sheets.

Merges data from the Albert roster and Google Form intake questionnaire to produce a consolidated student roster CSV file.

## Import dependencies


Import code dependencies

In [116]:
import numpy as np
import pandas as pd
from pathlib import Path

## Import data

Set up filenames of interest.

In [117]:
course_prefix = 'xx' # all file names will start with this prefix, e.g. 'wd', 'py', 'ad', 'se', etc.

In [118]:
# all relevant filenames for this course
roster_file = f'./rosters/{course_prefix}-roster.csv' # the roster file
SKIP_ROSTER_ROWS = 3 # number of rows in roster file to skip to get to column headings
questionnaire_file = f'./questionnaires/{course_prefix}-intake.csv' # the intake questionnaire file
result_file = f'./results/{course_prefix}-result.csv' # the output file we will generate

Import the student roster from Albert.

In [119]:
roster_df = pd.read_csv(Path(roster_file).resolve(), skiprows=SKIP_ROSTER_ROWS)

Clean up student roster.

In [120]:
# simplify some field names
field_prefixes = ['Email', 'First', 'Last'] # prefixes in field names we want to simplify
for prefix in field_prefixes:
    # rename any field name starting with this prefi to this prefix only
    roster_df.rename(columns=lambda x: x if not x.startswith(prefix) else prefix, inplace=True)
roster_df.columns

Index(['Counter', 'Campus ID', 'Last', 'First', 'Pronoun', 'Name Recording',
       'Email', 'Units Taken', 'Plan Description', 'Academic Level',
       'Student Location', 'Status', 'Status Notes'],
      dtype='object')

Import the student intake questionnaire results.

In [121]:
intake_df = pd.read_csv(Path(questionnaire_file).resolve())

Cleanup intake questionnaire data.

In [122]:
# rename field with name starting with 'Discord' to simply 'Discord'
field_prefixes = ['Discord', 'GitHub', 'Email', 'First', 'Last'] # prefixes in field names we want to simplify
for prefix in field_prefixes:
    # rename any field name starting with this prefi to this prefix only
    intake_df.rename(columns=lambda x: x if not x.startswith(prefix) else prefix, inplace=True)
intake_df.columns

Index(['Timestamp', 'Email', 'First', 'Discord', 'GitHub',
       'What are your motivations in taking this course? (You will not be judged in any way by your answers)',
       'What year are you in?',
       'What programming languages are you comfortable with?',
       'Any other comments you'd like to share or interests you'd like to express?',
       'Last'],
      dtype='object')

Cleanup roster columns.

In [123]:
# rename field with name starting with 'Discord' to simply 'Discord'
field_prefixes = ['Discord', 'GitHub', 'Email', 'First', 'Last'] # prefixes in field names we want to simplify
for prefix in field_prefixes:
    # rename any field name starting with this prefi to this prefix only
    roster_df.rename(columns=lambda x: x if not x.startswith(prefix) else prefix, inplace=True)
roster_df.columns

Index(['Counter', 'Campus ID', 'Last', 'First', 'Pronoun', 'Name Recording',
       'Email', 'Units Taken', 'Plan Description', 'Academic Level',
       'Student Location', 'Status', 'Status Notes'],
      dtype='object')

## Merge datasets
Create one master dataframe.

In [124]:
# join the two CSV files into a single dataframe
df = pd.merge(roster_df, intake_df, on='Email', how='left', suffixes=(None, '_intake'))
df.sample(3) # random sample

,Counter,Campus ID,Last,First,Pronoun,Name Recording,Email,Units Taken,Plan Description,Academic Level,...,Status Notes,Timestamp,First_intake,Discord,GitHub,What are your motivations in taking this course? (You will not be judged in any way by your answers),What year are you in?,What programming languages are you comfortable with?,Any other comments you'd like to share or interests you'd like to express?,Last_intake
1,4,N87654321,Doe,Jane,NaN,NaN,jdoe@myuni.edu,4,UA-Coll of Arts & Sci-Psychology,Senior,...,NaN,5/20/25 14:45,Lisa,lisa_king,lisaking567,Want to build my own apps and websites,Freshman,HTML,CSS,King
3,6,N99887766,Patel,Samir,NaN,NaN,spatel@myuni.edu,4,UB-Stern Schl Business-Ugrd-Finance,Junior,...,NaN,5/21/25 15:20,Chloe,chloem2345,chloemartin,To meet new people and collaborate on projects,Sophomore,Python,R,Martin
2,5,N11223344,Lee,Alex,NaN,NaN,alee@myuni.edu,2,UA-Coll of Arts & Sci-Mathematics,Freshman,...,NaN,5/21/25 08:30,Raj,rajtech,rajpatel9101,Curious about how technology works,Junior,Python,C++,Patel


Remove unnecessary columns.

In [125]:
# keep only a few important columns
df = df[['Last', 'First', 'Email', 'GitHub', 'Discord', ]]
df.head(5)

,Last,First,Email,GitHub,Discord
0,Smith,John,jsmith@myuni.edu,mbrown2025,mikeb123
1,Doe,Jane,jdoe@myuni.edu,lisaking567,lisa_king
2,Lee,Alex,alee@myuni.edu,rajpatel9101,rajtech
3,Patel,Samir,spatel@myuni.edu,chloemartin,chloem2345
4,Kim,Sarah,skim@myuni.edu,davidpark,davidp6789


Remove rows that do not have an email address.

In [126]:
# remove rows that have no email address
df = df[df['Email'].notna()]
# remove rows that do not have a valid email address
df = df[df['Email'].str.contains('@')]

## Save to CSV
Save the dataframe to CSV for later use.

In [127]:
# save df to a CSV file
output_path = Path(result_file).resolve()
df.to_csv(output_path, index=False)